# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [17]:
# Importar as libs
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [13]:
df = pd.read_csv('previsao_de_renda.csv')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [16]:
# Exercício 1: Ajustar um modelo com log(renda) usando variáveis categóricas com referência
df['log_renda'] = np.log(df['renda'])

# Ajustar o modelo inicial com todas as variáveis consideradas
modelo1 = smf.ols('log_renda ~ sexo + idade + tempo_emprego + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia + qtd_filhos + idade + tempo_emprego', data=df).fit()

# Mostrar o resumo
print(modelo1.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     299.5
Date:                Wed, 25 Jun 2025   Prob (F-statistic):               0.00
Time:                        23:35:00   Log-Likelihood:                -13571.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12403   BIC:                         2.737e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [18]:
# Exercício 2: Verificar multicolinearidade usando VIF

# Criar as matrizes y e X para cálculo do VIF
formula = 'log_renda ~ sexo + idade + tempo_emprego + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia + qtd_filhos + idade + tempo_emprego'
y, X = dmatrices(formula, data=df, return_type='dataframe')

# Calcular VIF para cada variável
vif = pd.DataFrame()
vif['variavel'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Mostrar os resultados
print(vif)

                            variavel         VIF
0                          Intercept  213.026436
1                          sexo[T.M]    1.186788
2           posse_de_veiculo[T.True]    1.165088
3            posse_de_imovel[T.True]    1.057263
4             tipo_renda[T.Bolsista]    1.004514
5           tipo_renda[T.Empresário]    1.078164
6          tipo_renda[T.Pensionista]    1.005027
7     tipo_renda[T.Servidor público]    1.079715
8          educacao[T.Pós graduação]    1.258783
9             educacao[T.Secundário]   30.371803
10     educacao[T.Superior completo]   29.297502
11   educacao[T.Superior incompleto]    6.190214
12          estado_civil[T.Separado]    1.044007
13          estado_civil[T.Solteiro]    1.088714
14             estado_civil[T.União]    1.040812
15             estado_civil[T.Viúvo]    1.053457
16           tipo_residencia[T.Casa]    6.800892
17    tipo_residencia[T.Com os pais]    4.458718
18    tipo_residencia[T.Comunitário]    1.357337
19        tipo_resid

In [19]:
# Exercício 3: Criar variáveis transformadas
# Criar idade ao quadrado
df['idade2'] = df['idade'] ** 2

# Criar uma variável categórica para idade (faixas)
df['faixa_idade'] = pd.cut(df['idade'], bins=[0, 25, 40, 60, 100], labels=['jovem', 'adulto', 'meia_idade', 'idoso'])

# Visualizar as transformações
print(df[['idade', 'idade2', 'faixa_idade']].head())


   idade  idade2 faixa_idade
0     26     676      adulto
1     28     784      adulto
2     35    1225      adulto
3     30     900      adulto
4     33    1089      adulto


In [21]:
# Exercício 4: Ajustar dois novos modelos com idade ao quadrado e com faixa etária
# Modelo com idade e idade²
modelo_quad = smf.ols('log_renda ~ idade + idade2 + sexo + tempo_emprego + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia + qtd_filhos', data=df).fit()
print('Modelo com idade²:')
print(modelo_quad.summary())

# Modelo com faixa etária (variável categórica)
modelo_faixa = smf.ols('log_renda ~ faixa_idade + sexo + tempo_emprego + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia + qtd_filhos', data=df).fit()
print('\nModelo com faixa etária:')
print(modelo_faixa.summary())


Modelo com idade²:
                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.0
Date:                Wed, 25 Jun 2025   Prob (F-statistic):               0.00
Time:                        23:36:35   Log-Likelihood:                -13571.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.738e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------

In [22]:
# Exercício 5: Comparar os modelos com base no AIC e R²-ajustado
print('Modelo inicial:')
print('R²-ajustado:', modelo1.rsquared_adj)
print('AIC:', modelo1.aic)

print('\nModelo com idade²:')
print('R²-ajustado:', modelo_quad.rsquared_adj)
print('AIC:', modelo_quad.aic)

print('\nModelo com faixa etária:')
print('R²-ajustado:', modelo_faixa.rsquared_adj)
print('AIC:', modelo_faixa.aic)


Modelo inicial:
R²-ajustado: 0.355896094676444
AIC: 27190.68676690226

Modelo com idade²:
R²-ajustado: 0.3558450760678109
AIC: 27192.669078556686

Modelo com faixa etária:
R²-ajustado: 0.35545649028953985
AIC: 27201.161336751204
